In [1]:
from up_map import Map
import random
import numpy as np
import copy
import timeit

In [10]:
"""
-Start with k randomly generated states (population) 

-Do until “done”
    Select k2<<k most fit states to be preserved (elitism)
    Remove k3<k weakest states from population (culling)
    Repeat 
        Select two states semi-randomly
            Weight towards states with better fitness
            Think of fitness as opposite of heuristic function
        Combine two states to generates two successors
        Randomly change some bits in the states (mutation)

    Until population is full

"""

class genetic(Map):
    def __init__(self,filename):
        """
        functions: get population; sort and get elitism and culling; crossover; mutation; 
        Running whole algorithm
        
        """
        Map.__init__(self,filename)
        self.get_map()
        
        self.during_time = 0
        self.time = 0 
        self.max_score = None
        
        self.size = 300
        self.elitism_size = 5
        self.culling_size = 10

        self.population = []       
        self.sort_population = []
        self.elitism = []
        self.culling = []
        self.children = []
        
        self.max_score = None
        self.best_state = None
        self.current_time = None
        
        self.results = []
        self.result = None
        
    def initial_population(self):
        for i in range(self.size):
            current_map = self.initial_map()
            self.population.append(current_map)
            
    def rank_population(self):
        temp = []
        for child in self.population:
            score = self.score(child)
            temp.append((score,child))
            
        # from small to large
        temp.sort(key=lambda x: x[0])
        
        for i in temp:
            self.sort_population.append(i[1])
        
       
    def get_elitism_culling(self):
        population = copy.deepcopy(self.sort_population)
        for i in range(self.elitism_size):
            self.elitism.append(population.pop())
            
        for i in range(self.culling_size):
            self.culling.append(self.sort_population.pop(0))
            
    def check_children(self,state):
        """
        input:
        array([[ 0.,  0.,  0.,  0.],
                [ 0., 12.,  0.,  0.],
                [ 0., 13.,  0.,  0.]])
        output: True/False
        """
        industrial_positions = np.asarray(np.where(state == 12)).T
        commercial_positions = np.asarray(np.where(state == 13)).T
        residential_positions = np.asarray(np.where(state == 14)).T
        if len(industrial_positions)<= self.industrial and len(commercial_positions)<= self.commercial and len(residential_positions)<= self.residential:
            return True
        else:
            return False
        
            
    def crossover(self):
        """
        need: self.sort_population
        randomly choose two parents, and generate self.chilren
        """
        cutpoint = int(self.column/2)
        while len(self.children) < self.size - self.elitism_size:
            num1 = np.random.randint(0, len(self.sort_population))
            num2 = np.random.randint(0, len(self.sort_population))
            if num1 != num2:
                parent1 = self.sort_population[num1]
                parent2 = self.sort_population[num2]

                child1 = np.concatenate((parent1[:,cutpoint:],parent2[:,:cutpoint]),axis=1)
                child2 = np.concatenate((parent2[:,cutpoint:],parent1[:,:cutpoint]),axis=1)
                if self.check_children(child1):
                    self.children.append(child1)
                    
                if len(self.children) == self.size - self.elitism_size:
                    break
                    
                if self.check_children(child2):
                    self.children.append(child2)

                if len(self.children) == self.size - self.elitism_size:
                    break
                    
    def mutation(self):
        """
        for n children, it has chance to mutate, return children  
        """
        
        for child in self.children:
            
            row1 = np.random.randint(0, self.row)
            row2 = np.random.randint(0, self.row)
            column1 = np.random.randint(0, self.column)
            column2 = np.random.randint(0, self.column)
            if self.map_board[row1,column1] != 10 or self.map_board[row1,column1] != 11 or self.map_board[row2,column2] != 10 or self.map_board[row2,column2] != 11:
                temp = child[row1,column1]
                child[row1,column1] = child[row2,column2]
                child[row2,column2] =temp
                
    def genetic_algorithm(self):
        """
        run genetic algorithm
        end state: time over 10 seconds
        
        """
        self.initial_population()
        start = timeit.default_timer()
        i = 0 
        while True:
#             print('going through '+str(i)+' generation')
            self.rank_population()
            self.get_elitism_culling()
            self.crossover()
            
            self.mutation()
            self.children.extend(self.elitism)
            
            self.best_state = max(self.children, key=lambda child: self.score(child))
            self.max_score= self.score(self.best_state)
            self.current_time = timeit.default_timer() - start
            self.results.append((self.max_score,-self.current_time,self.best_state))
            
            self.population = self.children
            
            end = timeit.default_timer()
            i = i+1
            if (end - start) > 9.5:
                
                break
#         small to large
        self.results.sort()
        self.result =  self.results.pop()
        self.during_time = end - start
        
            
              

In [11]:
ga = genetic('urban 2.txt')


In [12]:
result = ga.result
result

(35, -1.1854097650000028, array([[ 0.,  0.,  0.,  0.,  0.],
        [13.,  0., 14.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., 14.],
        [ 0.,  0.,  0.,  0.,  0.]]))

In [13]:
ga.during_time

10.122309365000007

In [ ]:
# ga.map_board

In [13]:
ga.map_board = ga.map_board.astype(str)

In [14]:
for row in range(ga.map_board.shape[0]):
    for col in range(ga.map_board.shape[1]):
        if ga.map_board[row, col] == '10':
            ga.map_board[row, col] = 'X'
        elif ga.map_board[row, col] == '11':
            ga.map_board[row, col] = 'S'

for i in range(result[2].shape[0]):
    for j in range(result[2].shape[1]):
        if result[2][i, j] == 12:
            ga.map_board[i, j] = 'I'
        elif result[2][i, j] == 13:
            ga.map_board[i, j] = "R"
        elif result[2][i, j] == 14:
            ga.map_board[i, j] = "C"

In [20]:
output = open("urban planning GA.txt", 'w')
print("The score for best map is: ",result[0], file=output)
print("The time that score was first achieved:", -result[1], file=output)
print("The map of the city:", file=output)
print(ga.map_board, file=output)
output.close()

In [15]:
ga.map_board

array([['2', 'C', '3', 'X', '6'],
       ['4', 'X', '3', '2', '3'],
       ['3', '1', '1', 'R', 'C'],
       ['7', '6', '5', '8', '5'],
       ['S', '6', 'S', '9', '1'],
       ['4', '7', '2', '6', '5']], dtype='<U21')

In [19]:
print("The score for best map is: ",result[0])
# print("The time that score was first achieved:", -result[1])

The score for best map is:  28


In [22]:
print('The score for this map: '+ str(result[0]))
print('The time that score was first achieved: '+ str(-result[1]))
print('Final map: '+str(result[2]))

The score for this map: 37
The time that score was first achieved: 3.2121822559993234
Final map: [[ 0. 13.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0. 13.  0.  0.]
 [14. 14.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


In [25]:
list1 = [(1,2,'1'),(1,1,'2'),(3,-1,'3'),(3,-2,'3')]
list1.sort()
list1

[(1, 1, '2'), (1, 2, '1'), (3, -2, '3'), (3, -1, '3')]

In [27]:
ga = genetic('urban 1.txt')


In [16]:
random.random()<0.2

False

In [18]:
random.random()

0.3698863059699782

In [16]:
for child in ga.children:

    row1 = np.random.randint(0, ga.row)
    row2 = np.random.randint(0, ga.row)
    column1 = np.random.randint(0, ga.column)
    column2 = np.random.randint(0, ga.column)
    if ga.map_board[row1,column1] != 10 or ga.map_board[row1,column1] != 11 or ga.map_board[row2,column2] != 10 or ga.map_board[row2,column2] != 11:

        temp = child[row1,column1]
        child[row1,column1] = child[row2,column2]
        child[row2,column2] =temp


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [14.  0.  0.  0.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [14.  0.  0.  0.]]
[[ 0.  0.  0. 13.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[ 0.  0.  0. 13.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. 14. 13.]]
[[ 0. 13.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. 14.  0.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. 14. 13.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. 14. 13.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [17]:
ga.children

[array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]), array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [14.,  0.,  0.,  0.]]), array([[ 0.,  0.,  0., 13.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]), array([[ 0., 13.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0., 14.,  0.]]), array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0., 14., 13.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])]

In [13]:
list1 = [1,2,3,4,5,5,6]
for j in list1 = [1,2,3,4,5,5,6]
for j in range(10):
    for k in range(10):
        
        if i == 0:
            break
    print(j):
    for k in range(10):
        
        if i == 0:
            break
    print(j)

0
1
2
3
4
5
6
7
8
9


In [87]:
ga.population

[array([[ 0., 12.,  0.,  0.],
        [13.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]]), array([[ 0.,  0.,  0., 14.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]]), array([[ 0.,  0.,  0.,  0.],
        [ 0., 14.,  0.,  0.],
        [12.,  0.,  0.,  0.]]), array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., 14.],
        [ 0.,  0.,  0.,  0.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]), array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., 14.],
        [12.,  0.,  0.,  0.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]), array([[ 0.,  0.,  0., 12.],
        [ 0.,  0.,  0.,  0.],
        [14.,  0.,  0.,  0.]])]

In [62]:
a = list1[0]
a

array([[ 0.,  0., 13.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [66]:
b = list1[2]
b

array([[ 0.,  0.,  0.,  0.],
       [14.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [64]:
column = int(a.shape[1]/2)
column

2

In [73]:
a1 = a[:,:3]
b1 = b[:,:3]
a2 = a[:,3:]
b2 = b[:,3:]
print(a1)
print(a2)
print(b2)

[[ 0.  0. 13.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]]


In [74]:
np.concatenate((a1,b2),axis=1)

array([[ 0.,  0., 13.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [75]:
np.concatenate((a2,b1),axis=1)

array([[ 0.,  0.,  0.,  0.],
       [ 0., 14.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [21]:
map = Map('urban 1.txt')
map.get_map()

board = map.map_board


print(board)

[[ 0.  0. 12.  0.]
 [13.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[10  1  2  4]
 [ 3  4 11  3]
 [ 6  0  2  3]]


In [33]:
initial = map.initial_map()
print(initial)

[[ 0.  0.  0.  0.]
 [ 0. 14.  0.  0.]
 [ 0.  0.  0.  0.]]


In [13]:
size = 2
def populate(my_map,board):
    population = []
    i = 0
    while True:
        temp_map = my_map.copy()
        random.shuffle(temp_map)
        a = temp_map.reshape(board.shape)
        b_1 = a == 12
        b_2 = a == 13
        b_3 = a == 14
        if 10 not in board[b_1] and 10 not in board[b_2] and 10 not in board[b_3]:
            population.append(a)
            i += 1
        if i == size:
            break
    return population

In [19]:
populate(initial,board)

[array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])]

In [23]:
random.shuffle(initial)
a = initial.reshape((3,4))
a

array([[ 0.,  0., 12.,  0.],
       [ 0.,  0., 12.,  0.],
       [ 0.,  0., 12.,  0.]])

In [24]:
b_1 = a == 12

In [25]:
b_1

array([[False, False,  True, False],
       [False, False,  True, False],
       [False, False,  True, False]])

In [26]:
board[b_1]

array([ 2, 11,  2])

In [28]:
b_2 = a == 13
b_2

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

In [29]:
board[b_2]

array([], dtype=int64)